1. Spróbuj skonwertować to do SQL


In [2]:
import xml.etree.ElementTree as ET
import os
import re
import pyprind
from collections import defaultdict
import osmcsv

ImportError: No module named pyprind

In [9]:
wroclaw = 'test.osm'

In [24]:

def get_root(fname):

    tree = ET.parse(fname)
    return tree.getroot()


root = get_root(wroclaw)

In [6]:
def count_tags(filename):
    tags = defaultdict(lambda: 0)
    for i in ET.iterparse(filename):
        tags[i[1].tag] += 1
    return tags



ower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')


def key_type(element, keys):
    if element.tag == "tag":
        if lower.search(element.get("k")):
           keys["lower"] += 1
        elif lower_colon.search(element.get("k")):
           keys["lower_colon"] += 1
        elif problemchars.search(element.get("k")):
           keys["problemchars"] += 1
        else:
           keys["other"] += 1
        
        
    return keys


In [26]:
tags = count_tags(wroclaw)

In [27]:
tags

defaultdict(<function __main__.count_tags.<locals>.<lambda>>,
            {'bounds': 1,
             'member': 2,
             'nd': 529,
             'node': 392,
             'osm': 1,
             'relation': 1,
             'tag': 242,
             'way': 54})

In [13]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import xml.etree.cElementTree as ET
import pprint
import re

"""
Your task is to explore the data a bit more.
Before you process the data and add it into your database, you should check the
"k" value for each "<tag>" and see if there are any potential problems.

We have provided you with 3 regular expressions to check for certain patterns
in the tags. As we saw in the quiz earlier, we would like to change the data
model and expand the "addr:street" type of keys to a dictionary like this:
{"address": {"street": "Some value"}}
So, we have to see if we have such tags, and if we have any tags with
problematic characters.

Please complete the function 'key_type', such that we have a count of each of
four tag categories in a dictionary:
  "lower", for tags that contain only lowercase letters and are valid,
  "lower_colon", for otherwise valid tags with a colon in their names,
  "problemchars", for tags with problematic characters, and
  "other", for other tags that do not fall into the other three categories.
See the 'process_map' and 'test' functions for examples of the expected format.
"""


lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')


def key_type(element, keys):
    if element.tag == "tag":
        if lower.search(element.get("k")):
           keys["lower"] += 1
        elif lower_colon.search(element.get("k")):
           keys["lower_colon"] += 1
        elif problemchars.search(element.get("k")):
           keys["problemchars"] += 1
        else:
           keys["other"] += 1
                
    return keys


def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)
    return keys


In [14]:
process_map(wroclaw)

{'lower': 261870, 'lower_colon': 241766, 'other': 132699, 'problemchars': 1}

In [30]:
def process_map_user(filename):
    users = set()
    for _, element in ET.iterparse(filename):
        user = element.get("amnety")
        if user:
            users.add(user) 

    return users

In [29]:
len(process_map_user(wroclaw))

0

In [18]:
expected = ["Ulica", "Aleja", "Plac"]

# UPDATE THIS VARIABLE
mapping = { "Ul": "Ulica",
            "Ul..": "Ulica",
            "Al.": "Aleja"
            }

def audit_location_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)


def is_feature_name(feature_name):
    return (elem.attrib['k'] == feature_name)


def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types


def update_name(name, mapping):
    last_word = re.compile("\S+$")
    last_word = last_word.findall(name)[0]
    name = name.replace(last_word, mapping[last_word])
      
    
    return name

   

In [22]:
st_types = audit(wroclaw)

In [23]:
st_types

defaultdict(set,
            {'Abramowskiego': {'Edwarda Abramowskiego'},
             'Akacjowa': {'Aleja Akacjowa'},
             'Alberta': {'Brata Alberta'},
             'Altanowa': {'Altanowa'},
             'Antoniego': {'Świętego Antoniego'},
             'Arctowskiego': {'Henryka Arctowskiego'},
             'Babimojska': {'Babimojska'},
             'Bacha': {'Jana Sebastiana Bacha'},
             'Banacha': {'Stefana Banacha'},
             'Bandtkiego': {'Jerzego Samuela Bandtkiego'},
             'Barbary': {'Świętej Barbary'},
             'Bardzka': {'Bardzka'},
             'Barlickiego': {'Norberta Barlickiego'},
             'Bartla': {'Kazimierza Bartla'},
             'Barycka': {'Barycka'},
             'Bałuckiego': {'Michała Bałuckiego'},
             'Becka': {'Pułkownika Józefa Becka'},
             'Belwederczyków': {'Belwederczyków'},
             'Bema': {'Generała Józefa Bema', 'Plac Generała Józefa Bema'},
             'Benedyktyńska': {'Benedyktyńska'},
 

In [ ]:
pprint.pprint(dict(st_types))

for st_type, ways in st_types.iteritems():
    for name in ways:
        better_name = update_name(name, mapping)
        print name, "=>", better_name
        if name == "West Lexington St.":
            assert better_name == "West Lexington Street"
        if name == "Baldwin Rd.":
            assert better_name == "Baldwin Road"

How many streets we have? 
What says us colon in those cases where it is?
Ulica and other typse
Kościół



In [ ]:
 <node id="1168111478" lat="51.1140188" lon="17.0529489" version="2" timestamp="2014-06-15T20:59:58Z" changeset="22952386" uid="2098272" user="maraf24">
    <tag k="addr:city" v="Wrocław"/>
    <tag k="addr:housenumber" v="2"/>
    <tag k="addr:street" v="Piwna"/>
    <tag k="amenity" v="kindergarten"/>
    <tag k="entrance" v="yes"/>
    <tag k="name" v="Przedszkole Sióstr Jadwiżanek &quot;Ziarenko&quot;"/>
  </node>

school 

brothel

# OSM to CSV convertion

In [14]:
osmcsv.process_map(wroclaw, validate=True)

AttributeError: 'dict' object has no attribute 'iteritems'

# SQL section

In [ ]:
table_creator = """CREATE TABLE nodes (
    id INTEGER PRIMARY KEY NOT NULL,
    lat REAL,
    lon REAL,
    user TEXT,
    uid INTEGER,
    version INTEGER,
    changeset INTEGER,
    timestamp TEXT
);

CREATE TABLE nodes_tags (
    id INTEGER,
    key TEXT,
    value TEXT,
    type TEXT,
    FOREIGN KEY (id) REFERENCES nodes(id)
);

CREATE TABLE ways (
    id INTEGER PRIMARY KEY NOT NULL,
    user TEXT,
    uid INTEGER,
    version TEXT,
    changeset INTEGER,
    timestamp TEXT
);

CREATE TABLE ways_tags (
    id INTEGER NOT NULL,
    key TEXT NOT NULL,
    value TEXT NOT NULL,
    type TEXT,
    FOREIGN KEY (id) REFERENCES ways(id)
);

CREATE TABLE ways_nodes (
    id INTEGER NOT NULL,
    node_id INTEGER NOT NULL,
    position INTEGER NOT NULL,
    FOREIGN KEY (id) REFERENCES ways(id),
    FOREIGN KEY (node_id) REFERENCES nodes(id)
);"""